O cadastro do experimento é realizado na plataforma de experimentação;

O SDK recebe o id do experimento, o dataFrame, a coluna a ser randomizada e, em seguida, atribui os valores das variantes na coluna variante, cadastrada previamente.

Após randomização, o lote de usuários randomizados é enviado para plataforma de experimentação através de uma função que ativa o envio de eventos em um possível gluejob

---

# Exemplo de uso

### Cadastro via requisição

In [ ]:
# {
# "hipotese"{
#     "sigla": "HP6"
#     "se": "o botão"
#     "mudar_de": "azul"
#     "para": "verde"
#     "entao": "aumentaremos conversão"}
# "mudanca"{
#     "objeto": "teste_modelo"
#     "valores": ['A', 'B']}
# "metrica"{
#     "sucesso": "suc"
#     "segurança": "seg1"
#     "direcionadora" ['dir1', 'dir2']}
# "publico"{
#     "tamanho_amostral": 1000
#     "audiencia": 0.5
#     "tempo_duracao_semanas": 2}
# }

#Retorna o id do experimento, ao utilizar, o experimento já traz todas configurações, como audiência e quantidade de variantes, por exemplo.
#Deve haver verificação de acesso

### Uso do randomizador

In [2]:
import pandas as pd
import numpy as np

id_usuario = np.arange(1, 10001)
df = pd.DataFrame({'id_usuario': id_usuario})
df['id_usuario'] = df.index
df['id_usuario'] = df['id_usuario'].astype(str)
df.head()

,id_usuario
0,0
1,1
2,2
3,3
4,4


In [3]:
from randomizacao import RandomizacaoBatch
teste_modelo = RandomizacaoBatch(123)
teste_modelo = teste_modelo.executa_randomizacao(df,'id')

In [4]:
teste_modelo.head()

,id_usuario,variante
0,0,B
1,1,A
2,2,A
3,3,A
4,4,A


In [21]:
contagem = teste_modelo.groupby('variante').id_usuario.nunique()
contagem

variante
A    4949
B    5051
Name: id_usuario, dtype: int64

In [24]:
proporcao = contagem / teste_modelo.id_usuario.nunique()
proporcao

variante
A    0.4949
B    0.5051
Name: id_usuario, dtype: float64

### Randomização estratificada

In [12]:
import pandas as pd
import numpy as np

id_usuario = np.arange(1, 10001)
df = pd.DataFrame({'id_usuario': id_usuario})
df['id_usuario'] = df.index
df['id_usuario'] = df['id_usuario'].astype(str)

# Criando a coluna de categoria
segmento = ['IA', 'IU', 'IP']
df['categoria'] = np.random.choice(segmento, size=len(df))

df.head()

,id_usuario,categoria
0,0,IU
1,1,IA
2,2,IP
3,3,IP
4,4,IA


In [13]:
from randomizacao import RandomizacaoBatch

randomiza_estratificado = RandomizacaoBatch(123)
df_randomizado = randomiza_estratificado.executa_randomizacao_estratificada(df, 'id_usuario', 'categoria')

In [14]:
df_randomizado.head()

,id_usuario,categoria,variante
0,1,IA,B
1,4,IA,A
2,5,IA,B
3,6,IA,A
4,12,IA,A


In [18]:
contagem = df_randomizado.groupby(['variante', 'categoria']).id_usuario.nunique()
contagem

variante  categoria
A         IA           1674
          IP           1718
          IU           1683
B         IA           1656
          IP           1580
          IU           1689
Name: id_usuario, dtype: int64

In [19]:
proporcao = contagem / df_randomizado.groupby('variante').id_usuario.nunique()
proporcao

variante  categoria
A         IA           0.329852
          IP           0.338522
          IU           0.331626
B         IA           0.336244
          IP           0.320812
          IU           0.342944
Name: id_usuario, dtype: float64

### Envio para Data Mesh

In [10]:
randomiza_estratificado.enviar_para_datamesh(df_randomizado)

Job iniciado com JobRunId: response JobID
